# COSMOSIS VALIDATION

**Description**: This notebook serves as a validation of the `CosmoSIS` pipeline. Here we shall test both methods that we have implemented:

* **CASE 1**: `CosmoSIS` is run with `Cobaya` under-the-hood, with the Boltzmann solver being run through `Cobaya`. 

* **CASE 2**: `CosmoSIS` is called completely independent from `Cobaya`, even for the Boltzmann solver calculations.

The difference between the two cases are explained in more detail in the README.md file within this folder. For each case, we will provide a fixed set of parameters and calculate the final $\chi2$ value of the likelihood, comparing it with the value obtained with the fiducial CLOE/Cobaya pipeline. 

In [ ]:
# General python imports

import numpy as np
import matplotlib.pyplot as plt
import time 
import os, sys

In [ ]:
# If you have clone the repository and open this notebook,
# this notebook should be in likelihood-implementation/notebooks
# Let's set the working directory to be likelihood-implementation

likelihood_path = os.path.realpath(os.path.join(os.getcwd(),'..'))
sys.path.insert(0, likelihood_path)
print('Setting as working directory: ', likelihood_path)

In [ ]:
# Matplotlib params set-up

%matplotlib inline
plt.rc('xtick',labelsize=16)
plt.rc('ytick',labelsize=16)
plt.rc('font',size=25)
plt.rc('axes', titlesize=26)
plt.rc('axes', labelsize=25)
plt.rc('lines', linewidth=2)
plt.rc('lines', markersize=6)
plt.rc('legend', fontsize=20)
plt.rc('mathtext', fontset='stix')
plt.rc('font', family='STIXGeneral')

`Cobaya` needs some modules to run: i.e: CAMB, CLASS, Polychord, other likelihood codes, and data (Planck 18, DES...)
If you have installed the CLOE as in the README instructions, you need to activate the conda environment `cloe` to run. This conda environment has CAMB installed, so you won't need to worry about anything else

Alternatively, if you have already installed Cobaya and other Cosmological codes such as CAMB, CLASS, Polychord, or Planck 18 you have 2 options:

* **(1)**:  point out where each of them is installed with the flag 'path' in the dictionary of cell 5. See the comment in the cell 5 corresponding to the 'theory' key of the 'info' dictionary


* **(2)**: if you installed the Cosmological codes as Cobaya automatic installation suggests  (https://cobaya.readthedocs.io/en/latest/installation_cosmo.html) you need to point out the path to your modules
as in the variable `modules_path` below:

In [ ]:
# ATTENTION: write down the path to your COBAYA modules if you want to follow option (2) above.
# Otherwise skip this cell
# modules_path': "YOUR_PATH_HERE"

## Create `Cobaya` benchmark

Here we first run the fiducial CLOE/Cobaya pipeline to obtain the $\chi^2$ value of one point. We do this by creating the `model` wrapper of `Cobaya` and making an evaluation of a single point (following the method of Case 2.1 in `DEMO.ipynb`)

In [ ]:
# Import external loglike from CLOE; that is, the class EuclidLikelihood within cobaya_interface.py

from cloe.cobaya_interface import EuclidLikelihood

In [ ]:
# We are running the Euclid-Likelihood as an external likelihood class for Cobaya
# Cobaya needs a dictionary or yaml file as input to start running
info = {
    #'params': Cobaya's protected key of the input dictionary. 
    # Includes the parameters that the user would like to sample over:
'params': {
        'ombh2': 0.022, #Omega density of baryons times the reduced Hubble parameter squared
        'omch2': 0.1202, #Omega density of cold dark matter times the reduced Hubble parameter squared
        'H0': 67.0, #Hubble parameter evaluated today (z=0) in km/s/Mpc
        'tau': 0.0925, #optical depth
        'mnu': 0.06, #  sum of the mass of neutrinos in eV
        'nnu': 3.046, #N_eff of relativistic species 
        'As': 2.12605e-9, #Amplitude of the primordial scalar power spectrum
        'ns': 0.96, # primordial power spectrum tilt 
        'w': -1.0, #Dark energy fluid model
        'wa': 0.0, #Dark energy fluid model
        'omk': 0.0, #curvature density
        'omegam': None, #DERIVED parameter: Omega matter density
        'omegab': None, #DERIVED parameter: Omega baryon density
        'omeganu': None, #DERIVED parameter: Omega neutrino density
        'omnuh2': None, #DERIVED parameter: Omega neutrino density times de reduced Hubble parameter squared
        'omegac': None, #DERIVED parameter: Omega cold dark matter density
        'N_eff': None},
    #'theory': Cobaya's protected key of the input dictionary.
    # Cobaya needs to ask some minimum theoretical requirements to a Boltzman Solver
    # (UC): you can choose between CAMB or CLASS
    # In this DEMO, we use CAMB and specify some CAMB arguments
    # such as the number of massive neutrinos
    # and the dark energy model
    #
    # ATTENTION: If you have CAMB/CLASS already installed and 
    # you are not using the likelihood conda environment 
    # or option (2) in cell (3) (Cobaya modules), you can add an extra key called 'path' within the camb dictionary
    # to point to your already installed CAMB code
    'theory': {'camb': 
               {'stop_at_error': True, 
                'extra_args':{'num_massive_neutrinos': 1,
                              'dark_energy_model': 'ppf',
                              # 'AccuracyBoost': 2.0
                             }}},
    #'sampler': Cobaya's protected key of the input dictionary.
    # In this DEMO, we use the 'evaluate' sampler to make a single computation of the posterior distributions
    'sampler': {'evaluate': None},  
    # 'packages_path': Cobaya's protected key of the input dictionary.
    # This is the variable you need to update
    # if you are running Cobaya with cobaya_modules (option (2) above).
    # If you are using the conda likelihood environment or option (1),
    # please, keep the line below commented
    #
    #'packages_path': modules_path,
    #
    #'output': Cobaya's protected key of the input dictionary.
    # Where are the results going to be stored, in case that the sampler produce output files? 
    # For example: chains...
    # (UC): modify the path below within 'output' to choose a name and a directory for those files
    'output': 'chains/cobaya_benchmark',
    #'debug': Cobaya's protected key of the input dictionary.
    # (UC): how much information you want Cobaya to print? If debug: True, it prints every single detail
    # that is going on internally in Cobaya
    'debug': False,
    #'timing': Cobaya's protected key of the input dictionary.
    # (UC): if timing: True, Cobaya returns how much time it took it to make a computation of the posterior
    # and how much time take each of the modules to perform their tasks
    'timing': False,
    #'force': Cobaya's protected key of the input dictionary.
    # (UC): if 'force': True, Cobaya forces deleting the previous output files, if found, with the same name
    'force': True
    }

In [ ]:
#'likelihood': Cobaya's protected key of the input dictionary.
# In this DEMO, we load the Euclid-Likelihood as an external function, and name it 'Euclid'
info['likelihood'] = {'Euclid': 
                     {'external': EuclidLikelihood, # Likelihood Class to be read as external
                     # Note: everything down below will overwrite the information read
                     # in the config folder
                     #
                     # Select which observables want to use during the analysis
                     # by setting them to True or False
                     'observables_selection': {
                         'WL': {'WL': True, 'GCphot': False, 'GCspectro': False},
                         'GCphot': {'GCphot': False, 'GCspectro': False},
                         'GCspectro': {'GCspectro': False}
                     },
                     # Plot the selected observables matrx
                     'plot_observables_selection': False,  
                     # Bias model
                     'bias_model': 1,
                     # Magnification bias model
                     # 1 -> linear interpolation
                     # 2 -> constant within bins
                     'magbias_model': 2,
                     # Switch to allow for matrix transformations of theory and data vectors
                     'matrix_transform_phot' : False, # 'BNT' or 'BNT-test,
                     # Nonlinear flags
                     # With this, the user can specify which nonlinear model they want
                     # For the time-being the available options are: 
                     # NL_flag_phot_matter
                     # 0 -> linear-only
                     # 1 -> Takahashi
                     # 2 -> Mead2016 (includes baryon corrections)
                     # 3 -> Mead2020 (w/o baryon corrections)
                     # 4 -> Mead2020_feedback (includes baryon corrections)
                     # 5 -> EE2
                     # 6 -> Bacco (matter)
                     'NL_flag_phot_matter': 0,
                     # NL_flag_spectro
                     # 0 -> linear-only
                     # 1 -> EFT
                     'NL_flag_spectro': 0,
                     'solver': 'camb',
                     'params': {
                        # Intrinsic alignment
                        'aia': 1.72,
                        'bia': 0.0,
                        'nia': -0.41,
                        # Photometric galaxy bias
                        'b1_photo': 1.0997727037892875,
                        'b2_photo': 1.220245876862528,
                        'b3_photo': 1.2723993083933989,
                        'b4_photo': 1.316624471897739,
                        'b5_photo': 1.35812370570578,
                        'b6_photo': 1.3998214171814918,
                        'b7_photo': 1.4446452851824907,
                        'b8_photo': 1.4964959071110084,
                        'b9_photo': 1.5652475842498528,
                        'b10_photo': 1.7429859437184225,
                        # Photometric galaxy bias (polynomial case)
                        'b0_poly_photo': 0.830703,
                        'b1_poly_photo': 1.190547,
                        'b2_poly_photo': -0.928357,
                        'b3_poly_photo': 0.423292,
                        # Magnification bias
                        'magnification_bias_1': 0.0,
                        'magnification_bias_2': 0.0,
                        'magnification_bias_3': 0.0,
                        'magnification_bias_4': 0.0,
                        'magnification_bias_5': 0.0,
                        'magnification_bias_6': 0.0,
                        'magnification_bias_7': 0.0,
                        'magnification_bias_8': 0.0,
                        'magnification_bias_9': 0.0,
                        'magnification_bias_10': 0.0,
                        # Multiplicative bias
                        'multiplicative_bias_1': 0.0,
                        'multiplicative_bias_2': 0.0,
                        'multiplicative_bias_3': 0.0,
                        'multiplicative_bias_4': 0.0,
                        'multiplicative_bias_5': 0.0,
                        'multiplicative_bias_6': 0.0,
                        'multiplicative_bias_7': 0.0,
                        'multiplicative_bias_8': 0.0,
                        'multiplicative_bias_9': 0.0,
                        'multiplicative_bias_10': 0.0,
                        # Uncertainty in photometric bins
                        'dz_1_GCphot': 0.0, 'dz_1_WL': 0.0,
                        'dz_2_GCphot': 0.0, 'dz_2_WL': 0.0,
                        'dz_3_GCphot': 0.0, 'dz_3_WL': 0.0,
                        'dz_4_GCphot': 0.0, 'dz_4_WL': 0.0,
                        'dz_5_GCphot': 0.0, 'dz_5_WL': 0.0,
                        'dz_6_GCphot': 0.0, 'dz_6_WL': 0.0,
                        'dz_7_GCphot': 0.0, 'dz_7_WL': 0.0,
                        'dz_8_GCphot': 0.0, 'dz_8_WL': 0.0,
                        'dz_9_GCphot': 0.0, 'dz_9_WL': 0.0,
                        'dz_10_GCphot': 0.0, 'dz_10_WL': 0.0,
                        # Growth rate index for scenario different from GR
                        'gamma_MG': 0.55,
                        # Spectroscopic galaxy bias
                        'b1_spectro_bin1': 1.46,
                        'b1_spectro_bin2': 1.61,
                        'b1_spectro_bin3': 1.75,
                        'b1_spectro_bin4': 1.9,
                        'b2_spectro_bin1': 0.0,
                        'b2_spectro_bin2': 0.0,
                        'b2_spectro_bin3': 0.0,
                        'b2_spectro_bin4': 0.0,
                        # Finger of God counterterms
                        'c0_spectro_bin1': 0.0,
                        'c0_spectro_bin2': 0.0,
                        'c0_spectro_bin3': 0.0,
                        'c0_spectro_bin4': 0.0,
                        'c2_spectro_bin1': 0.0,
                        'c2_spectro_bin2': 0.0,
                        'c2_spectro_bin3': 0.0,
                        'c2_spectro_bin4': 0.0,
                        'c4_spectro_bin1': 0.0,
                        'c4_spectro_bin2': 0.0,
                        'c4_spectro_bin3': 0.0,
                        'c4_spectro_bin4': 0.0,
                        # Shot noise parameters
                        'aP_spectro_bin1': 0.0,
                        'aP_spectro_bin2': 0.0,
                        'aP_spectro_bin3': 0.0,
                        'aP_spectro_bin4': 0.0,
                        'Psn_spectro_bin1': 0.0,
                        'Psn_spectro_bin2': 0.0,
                        'Psn_spectro_bin3': 0.0,
                        'Psn_spectro_bin4': 0.0,
                        # Purity of spectroscopic samples
                        'f_out': 0.0,
                        'f_out_1': 0.0,
                        'f_out_2': 0.0,
                        'f_out_3': 0.0,
                        'f_out_4': 0.0}, 
                     # k values for extrapolation of the matter power spectrum and size k-array
                     'k_max_extrap': 500.0,
                     'k_min_extrap': 0.0001,   
                     'k_samp': 1000,
                     # z limit values and size z-array
                     'z_min': 0.0,
                     'z_max': 4.0,
                     'z_samp': 100,
                     # Add RSD to photometric probes
                     'add_phot_RSD': False,
                     # Use MG gamma
                     'use_gamma_MG': False,
                     # Redshift dependent purity correction
                     'f_out_z_dep': False,
                     #'data': This give specifications for the paths of the input data files
                     'data': { 
                        #'sample' specifies the first folder below the main data folder
                        'sample': 'ExternalBenchmark',
                        #'spectro' and 'photo' specify paths to data files.
                        'spectro': {
                            # GC Spectro root name should contain z{:s} string
                            # to enable iteration over bins
                            'root': 'cov_power_galaxies_dk0p004_z{:s}.fits',
                            'redshifts': ["1.", "1.2", "1.4", "1.65"],
                            'edges': [0.9, 1.1, 1.3, 1.5, 1.8],
                            'cov_is_num': False,
                            'cov_nsim': 3500},
                        'photo': {
                            'redshifts': [0.2095, 0.489, 0.619, 0.7335, 0.8445, 0.9595, 1.087, 1.2395, 1.45, 2.038],
                            'ndens_GC': 'niTab-EP10-RB00.dat',
                            'ndens_WL': 'niTab-EP10-RB00.dat',
                            'luminosity_ratio': 'luminosity_ratio.dat',
                            # Photometric root names should contain z{:s} string
                            # to specify IA model
                            'root_GC': 'Cls_{:s}_PosPos.dat',
                            'root_WL': 'Cls_{:s}_ShearShear.dat',
                            'root_XC': 'Cls_{:s}_PosShear.dat',
                            'IA_model': 'zNLA',
                            # Photometric covariances root names should contain z{:s} string
                            # to specify how the covariance was calculated
                            'cov_GC': 'CovMat-PosPos-{:s}-20Bins.npy',
                            'cov_WL': 'CovMat-ShearShear-{:s}-20Bins.npy',
                            'cov_3x2pt': 'CovMat-3x2pt-{:s}-20Bins.npy',
                            'cov_model': 'Gauss',
                            'cov_is_num': False,
                            'cov_nsim': 10000}}, 
                    }}

In [ ]:
from cloe.auxiliary.likelihood_yaml_handler import set_halofit_version
# Set the correct halofit version under the 'extra_args' field in ['theory']['camb']
set_halofit_version(info, info['likelihood']['Euclid']['NL_flag_phot_matter'])
# Now we are ready to run Cobaya

In [ ]:
# First: import model wrapper of Cobaya 
from cobaya.model import get_model
t1 = time.time()
# Second: create an instance of the `model` wrapper called model
model = get_model(info)
logposterior = model.logposterior({})
cobaya_loglike = logposterior.loglikes

# Create an instance of the class EuclidLikelihood
like = EuclidLikelihood()
# Initialize default parameters for redshift, k-array, fiducial cosmology...
like.initialize()
# Get the cosmo_dictionary where all the cosmology + theory parameters are saved
like.passing_requirements(model, info, **model.provider.params)
# Update the cosmology dictionary with interpolators + basic quantities such as
# P_gg, P_delta...
like.cosmo.update_cosmo_dic(like.cosmo.cosmo_dic['z_win'], 0.05)
print('Time for calculation of the likelihood (Cobaya Benckmark): ', time.time()-t1)
print('   derived params: %r' % dict(zip(list(model.parameterization.derived_params()), logposterior.derived)))

# CASE 1: Run `CosmoSIS` with `Cobaya` under-the-hood
**User-case**: Here, we run the test case whereby `CosmoSIS` essentially calls `Cobaya` under-the-hood to calculate the cosmological parameters and matter power spectra. This is the equivalent to running
```bash
cosmosis run_cosmosis_with_cobaya.ini
```
in command line, except through a Python notebook.

In [ ]:
# Import cosmosis
import cosmosis

In [ ]:
# Setup the relevant CLOE module, based on the cosmosis_interface.py script
%run -i 'cosmosis_interface.py'

euclid_module = cosmosis.FunctionModule('euclid', setup, execute)
# Pass on parameters under the [euclid] section in the ini file as a dictionary
params = {
    'euclid': {
        'config_file': 'cobaya_config_for_cosmosis.yaml',
        'cobaya_output': 'chains/case_A/cobaya_output'
    }
}
euclid_module.setup(params)

In [ ]:
#Define fixed set of parameter values as a dictionary
values = {
    'cosmological_parameters': {
        'ombh2': 0.022,
        'omch2': 0.1202,
        'H0': 67.0,
        'tau': 0.0925,
        'mnu': 0.06,
        'nnu': 3.046,
        'As': 2.12605e-9,
        'ns': 0.96,
        'w': -1.0,
        'wa': 0.0,
        'omk': 0.0,
        # Intrinsic alignment
        'aia': 1.72,
        'bia': 0.0,
        'nia': -0.41,
        # Photometric galaxy bias
        'b1_photo': 1.0997727037892875,
        'b2_photo': 1.220245876862528,
        'b3_photo': 1.2723993083933989,
        'b4_photo': 1.316624471897739,
        'b5_photo': 1.35812370570578,
        'b6_photo': 1.3998214171814918,
        'b7_photo': 1.4446452851824907,
        'b8_photo': 1.4964959071110084,
        'b9_photo': 1.5652475842498528,
        'b10_photo': 1.7429859437184225,
        # Photometric galaxy bias (polynomial case)
        'b0_poly_photo': 0.830703,
        'b1_poly_photo': 1.190547,
        'b2_poly_photo': -0.928357,
        'b3_poly_photo': 0.423292,
        # Magnification bias
        'magnification_bias_1': 0.0,
        'magnification_bias_2': 0.0,
        'magnification_bias_3': 0.0,
        'magnification_bias_4': 0.0,
        'magnification_bias_5': 0.0,
        'magnification_bias_6': 0.0,
        'magnification_bias_7': 0.0,
        'magnification_bias_8': 0.0,
        'magnification_bias_9': 0.0,
        'magnification_bias_10': 0.0,
        # Multiplicative bias
        'multiplicative_bias_1': 0.0,
        'multiplicative_bias_2': 0.0,
        'multiplicative_bias_3': 0.0,
        'multiplicative_bias_4': 0.0,
        'multiplicative_bias_5': 0.0,
        'multiplicative_bias_6': 0.0,
        'multiplicative_bias_7': 0.0,
        'multiplicative_bias_8': 0.0,
        'multiplicative_bias_9': 0.0,
        'multiplicative_bias_10': 0.0,
        # Uncertainty in photometric bins
        'dz_1_GCphot': 0.0, 'dz_1_WL': 0.0,
        'dz_2_GCphot': 0.0, 'dz_2_WL': 0.0,
        'dz_3_GCphot': 0.0, 'dz_3_WL': 0.0,
        'dz_4_GCphot': 0.0, 'dz_4_WL': 0.0,
        'dz_5_GCphot': 0.0, 'dz_5_WL': 0.0,
        'dz_6_GCphot': 0.0, 'dz_6_WL': 0.0,
        'dz_7_GCphot': 0.0, 'dz_7_WL': 0.0,
        'dz_8_GCphot': 0.0, 'dz_8_WL': 0.0,
        'dz_9_GCphot': 0.0, 'dz_9_WL': 0.0,
        'dz_10_GCphot': 0.0, 'dz_10_WL': 0.0,
        # Growth rate index for scenario different from GR
        'gamma_MG': 0.55,
        # Spectroscopic galaxy bias
        'b1_spectro_bin1': 1.46,
        'b1_spectro_bin2': 1.61,
        'b1_spectro_bin3': 1.75,
        'b1_spectro_bin4': 1.9,
        'b2_spectro_bin1': 0.0,
        'b2_spectro_bin2': 0.0,
        'b2_spectro_bin3': 0.0,
        'b2_spectro_bin4': 0.0,
        # Finger of God counterterms
        'c0_spectro_bin1': 0.0,
        'c0_spectro_bin2': 0.0,
        'c0_spectro_bin3': 0.0,
        'c0_spectro_bin4': 0.0,
        'c2_spectro_bin1': 0.0,
        'c2_spectro_bin2': 0.0,
        'c2_spectro_bin3': 0.0,
        'c2_spectro_bin4': 0.0,
        'c4_spectro_bin1': 0.0,
        'c4_spectro_bin2': 0.0,
        'c4_spectro_bin3': 0.0,
        'c4_spectro_bin4': 0.0,
        # Shot noise parameters
        'aP_spectro_bin1': 0.0,
        'aP_spectro_bin2': 0.0,
        'aP_spectro_bin3': 0.0,
        'aP_spectro_bin4': 0.0,
        'Psn_spectro_bin1': 0.0,
        'Psn_spectro_bin2': 0.0,
        'Psn_spectro_bin3': 0.0,
        'Psn_spectro_bin4': 0.0,
        # Purity of spectroscopic samples
        'f_out': 0.0,
        'f_out_1': 0.0,
        'f_out_2': 0.0,
        'f_out_3': 0.0,
        'f_out_4': 0.0
    }
}

In [ ]:
# Setup the CosmoSIS pipeline with the modules, their parameters and the values to be sampled
modules = [euclid_module]
pipe = cosmosis.LikelihoodPipeline(params, values=values, modules=modules)

In [ ]:
t2 = time.time()
# Get the likelihood value from the current set of parameters
cosmosis_like, extra = pipe.likelihood(params)
# Define the sampler. We will use the test sampler to evaulate the logposterior of a single point
sampler_params = {
    "test": {
        # save all output to a directory 
        'save_dir': 'chains/case_A/cosmosis_output'
    }
}
# Make a Test sampler
sampler = cosmosis.samplers.TestSampler(sampler_params, pipe)
# The config method sets up the sampler.  It assumes the pipeline has already been configured.
sampler.config()
# Execute the sampler
sampler.execute()

print('Time for calculation of the likelihood (Cosmosis Case A): ', time.time()-t2)

## Comparison of $\chi^2$ value

In [ ]:
# Finally, we compare the chi^2 value obtained with the Cobaya benchmark value
chi2_cobaya = -2 * cobaya_loglike[0]
chi2_cosmosis = -2 * cosmosis_like
print('chi^2 of Cobaya benchmark = ', chi2_cobaya)
print('chi^2 from CosmoSIS pipeline Case A = ', chi2_cosmosis)
print('Fractional Difference = ', (chi2_cobaya-chi2_cosmosis)/chi2_cobaya)

# Case 2: Run `CosmoSIS` independent of `Cobaya`

**User-case**: We test the second pipeline where `CosmoSIS` is being called independently of `Cobaya`. This is the equivalent to running
```bash
cosmosis run_cosmosis.ini
```
in command line, except through a Python notebook.

In [ ]:
# PLEASE DEFINE THE DIRECTORY OF THE COSMOSIS STANDARD LIBRARY ON YOUR SYSTEM
cosmosis_lib_dir = '/n06data/lgoh/scratch/cosmosis-standard-library/'

camb_file = cosmosis_lib_dir + 'boltzmann/camb/camb_interface.py'

In [ ]:
# Setup the CAMB module, based on the camb_interface.py script in the CosmoSIS public repo. 
%run -i $camb_file
camb_module = cosmosis.FunctionModule('camb',setup, execute)

# Setup the relevant CLOE module, based on the cosmosis_interface.py script
%run -i 'cloe_interface.py'
euclid_module = cosmosis.FunctionModule('euclid', setup, execute)

In [ ]:
# Pass on parameters under the [camb] and [euclid] section in the ini file as a dictionary
params = {
    'camb': {
        'mode': 'power',
        'feedback': 0,
        'kmin': 0.0001 / 0.67,
        'kmax_extrapolate': 500.0 / 0.67 ,
        'nk': 1000,
        'zmin': 0.0,
        'zmax': 4.0,
        'nz': 100,
        'nonlinear': 'none',
        # 'AccuracyBoost': 2.0,
        'use_ppf_w': True,
        'power_spectra': 'delta_tot weyl delta_nonu'
        },
    'euclid': {
        'config_file': 'cobaya_config_for_cosmosis.yaml'
        }
}
camb_module.setup(params)
euclid_module.setup(params)

In [ ]:
#Define fixed set of parameter values as a dictionary
values = {
    'cosmological_parameters': {
        'ombh2': 0.022,
        'omch2': 0.1202,
        'h0': 0.67,
        'tau': 0.0925,
        'mnu': 0.06,
        'nnu': 3.046,
        'A_s': 2.12605e-9,
        'n_s': 0.96,
        'w': -1.0,
        'wa': 0.0,
        'omega_k': 0.0
        },
    'cloe_parameters':{
        # Intrinsic alignment
        'aia': 1.72,
        'bia': 0.0,
        'nia': -0.41,
        # Photometric galaxy bias
        'b1_photo': 1.0997727037892875,
        'b2_photo': 1.220245876862528,
        'b3_photo': 1.2723993083933989,
        'b4_photo': 1.316624471897739,
        'b5_photo': 1.35812370570578,
        'b6_photo': 1.3998214171814918,
        'b7_photo': 1.4446452851824907,
        'b8_photo': 1.4964959071110084,
        'b9_photo': 1.5652475842498528,
        'b10_photo': 1.7429859437184225,
        # Photometric galaxy bias (polynomial case)
        'b0_poly_photo': 0.830703,
        'b1_poly_photo': 1.190547,
        'b2_poly_photo': -0.928357,
        'b3_poly_photo': 0.423292,
        # Magnification bias
        'magnification_bias_1': 0.0,
        'magnification_bias_2': 0.0,
        'magnification_bias_3': 0.0,
        'magnification_bias_4': 0.0,
        'magnification_bias_5': 0.0,
        'magnification_bias_6': 0.0,
        'magnification_bias_7': 0.0,
        'magnification_bias_8': 0.0,
        'magnification_bias_9': 0.0,
        'magnification_bias_10': 0.0,
        # Multiplicative bias
        'multiplicative_bias_1': 0.0,
        'multiplicative_bias_2': 0.0,
        'multiplicative_bias_3': 0.0,
        'multiplicative_bias_4': 0.0,
        'multiplicative_bias_5': 0.0,
        'multiplicative_bias_6': 0.0,
        'multiplicative_bias_7': 0.0,
        'multiplicative_bias_8': 0.0,
        'multiplicative_bias_9': 0.0,
        'multiplicative_bias_10': 0.0,
        # Uncertainty in photometric bins
        'dz_1_GCphot': 0.0, 'dz_1_WL': 0.0,
        'dz_2_GCphot': 0.0, 'dz_2_WL': 0.0,
        'dz_3_GCphot': 0.0, 'dz_3_WL': 0.0,
        'dz_4_GCphot': 0.0, 'dz_4_WL': 0.0,
        'dz_5_GCphot': 0.0, 'dz_5_WL': 0.0,
        'dz_6_GCphot': 0.0, 'dz_6_WL': 0.0,
        'dz_7_GCphot': 0.0, 'dz_7_WL': 0.0,
        'dz_8_GCphot': 0.0, 'dz_8_WL': 0.0,
        'dz_9_GCphot': 0.0, 'dz_9_WL': 0.0,
        'dz_10_GCphot': 0.0, 'dz_10_WL': 0.0,
        # Growth rate index for scenario different from GR
        'gamma_MG': 0.55,
        # Spectroscopic galaxy bias
        'b1_spectro_bin1': 1.46,
        'b1_spectro_bin2': 1.61,
        'b1_spectro_bin3': 1.75,
        'b1_spectro_bin4': 1.9,
        'b2_spectro_bin1': 0.0,
        'b2_spectro_bin2': 0.0,
        'b2_spectro_bin3': 0.0,
        'b2_spectro_bin4': 0.0,
        # Finger of God counterterms
        'c0_spectro_bin1': 0.0,
        'c0_spectro_bin2': 0.0,
        'c0_spectro_bin3': 0.0,
        'c0_spectro_bin4': 0.0,
        'c2_spectro_bin1': 0.0,
        'c2_spectro_bin2': 0.0,
        'c2_spectro_bin3': 0.0,
        'c2_spectro_bin4': 0.0,
        'c4_spectro_bin1': 0.0,
        'c4_spectro_bin2': 0.0,
        'c4_spectro_bin3': 0.0,
        'c4_spectro_bin4': 0.0,
        # Shot noise parameters
        'aP_spectro_bin1': 0.0,
        'aP_spectro_bin2': 0.0,
        'aP_spectro_bin3': 0.0,
        'aP_spectro_bin4': 0.0,
        'Psn_spectro_bin1': 0.0,
        'Psn_spectro_bin2': 0.0,
        'Psn_spectro_bin3': 0.0,
        'Psn_spectro_bin4': 0.0,
        # Purity of spectroscopic samples
        'f_out': 0.0,
        'f_out_1': 0.0,
        'f_out_2': 0.0,
        'f_out_3': 0.0,
        'f_out_4': 0.0
    }
}

In [ ]:
# Setup the CosmoSIS pipeline with the modules, their parameters and the values to be sampled
modules = [camb_module,euclid_module]
pipe = cosmosis.LikelihoodPipeline(params, values=values, modules=modules)

In [ ]:
t3 = time.time()
# Get the likelihood value from the current set of parameters
like_cosmosis, extra = pipe.likelihood(params)
# Define the sampler. We will use the test sampler to evaulate the logposterior of a single point
sampler_params = {
    "test": {
        # save all output to a directory 
        'save_dir': 'chains/case_B/cosmosis_output'
    }
}
# Make a Test sampler
sampler = cosmosis.samplers.TestSampler(sampler_params, pipe)
# The config method sets up the sampler.  It assumes the pipeline has already been configured.
sampler.config()
# Execute the sampler
sampler.execute()

print('Time for calculation of the likelihood (Cosmosis Case B): ', time.time()-t3)

## Comparison of $\chi^2$ value

In [ ]:
# Finally, we compare the chi^2 value obtained with the Cobaya benchmark value
chi2_cosmosis = -2 * like_cosmosis
print('chi^2 of Cobaya benchmark = ', chi2_cobaya)
print('chi^2 from CosmoSIS pipeline Case B = ', chi2_cosmosis)
print('Fractional Difference = ', (chi2_cobaya-chi2_cosmosis)/chi2_cobaya)

## Plot internal quantities and cosmological observables

In [ ]:
import seaborn as sns
import matplotlib
matplotlib.rcParams['text.usetex'] = False
sns.set_theme(style="ticks")
sns.set_palette(sns.color_palette("Paired"))
%matplotlib inline

plt.rc('xtick',labelsize=20)
plt.rc('ytick',labelsize=20)
plt.rc('font',size=20)
plt.rc('axes', titlesize=25)
plt.rc('axes', labelsize=20)
plt.rc('lines', linewidth=3)
plt.rc('lines', markersize=6)
plt.rc('legend', fontsize=20)

In [ ]:
# Let's plot the background quantities of cosmo_dic
# See http://pf-ist-likelihood.pages.euclid-sgs.uk/likelihood-implementation/likelihood.cosmo.cosmology.html
# for extra information

fig, axs = plt.subplots(2, 2, figsize=(13,10))
for ax in axs.flatten():
    ax.tick_params(axis='both',which='both',direction='in')
fig.suptitle('Background quantities')

cosmosis_output_dir = 'chains/case_B/cosmosis_output/'
z = np.loadtxt(cosmosis_output_dir+'growth_parameters/z.txt')

axs[0, 0].plot(like.cosmo.cosmo_dic['z_win'], like.cosmo.cosmo_dic['H_z_func'](like.cosmo.cosmo_dic['z_win']),
              label='Cobaya Benchmark')
axs[0, 0].plot(z, np.loadtxt(cosmosis_output_dir+'growth_parameters/h.txt'),':',c = 'k',label='CosmoSIS')
axs[0, 0].set_xlabel(r'$z$')
axs[0, 0].set_ylabel(r'$H(z)$ $[\rm km \times s^{-1} Mpc^{-1}]$')
axs[0, 0].legend(frameon=False)

axs[0, 1].plot(like.cosmo.cosmo_dic['z_win'], like.cosmo.cosmo_dic['d_z_func'](like.cosmo.cosmo_dic['z_win']),
              label = r'$D_{\rm{A}}(z)$ $[\rm{Mpc}]$')
axs[0, 1].plot(z, np.loadtxt(cosmosis_output_dir+'growth_parameters/da.txt'),':',c = 'k')
axs[0, 1].plot(like.cosmo.cosmo_dic['z_win'], like.cosmo.cosmo_dic['r_z_func'](like.cosmo.cosmo_dic['z_win']),
              label = r'$r(z)$')
axs[0, 1].plot(z, np.loadtxt(cosmosis_output_dir+'distances/d_c.txt'),':',c = 'k')
axs[0, 1].set_xlabel(r'$z$')
axs[0, 1].legend(frameon=False)

axs[1, 0].plot(like.cosmo.cosmo_dic['z_win'], like.cosmo.cosmo_dic['sigma8_z_func'](like.cosmo.cosmo_dic['z_win']),
              label = r'$\sigma_8(z)$')
axs[1, 0].plot(z, np.loadtxt(cosmosis_output_dir+'growth_parameters/sigma_8.txt'),':',c = 'k')
axs[1, 0].plot(like.cosmo.cosmo_dic['z_win'], like.cosmo.cosmo_dic['fsigma8_z_func'](like.cosmo.cosmo_dic['z_win']),
              label = r'$f\sigma_8(z)$')
axs[1, 0].plot(z, np.loadtxt(cosmosis_output_dir+'growth_parameters/fsigma_8.txt'),':',c = 'k')
axs[1, 0].set_xlabel(r'$z$')
axs[1, 0].legend(frameon=False)

axs[1, 1].plot(like.cosmo.cosmo_dic['z_win'], like.cosmo.cosmo_dic['D_z_k'],
              label = r'$D(z)$')
axs[1, 1].plot(z, np.loadtxt(cosmosis_output_dir+'growth_parameters/d_z.txt'),':',c = 'k')
axs[1, 1].plot(like.cosmo.cosmo_dic['z_win'], 
               like.cosmo.cosmo_dic['f_z'](like.cosmo.cosmo_dic['z_win']),
              label = r'$f(z) = f\sigma_8/\sigma_8$')
axs[1, 1].plot(z, np.loadtxt(cosmosis_output_dir+'growth_parameters/f_z.txt'),':',c = 'k')
axs[1, 1].set_xlabel(r'$z$')
axs[1, 1].legend(frameon=False)

plt.tight_layout()
plt.subplots_adjust(top=0.90)


In [ ]:
# Let's plot the matter power spectrum 
k_label           = r'$k$ $[\rm{Mpc}^{-1}]$'
pm_label          = r'$P_{\delta\delta} \left[\rm{Mpc}^3\right]$'

# WARNING: AT THE MOMENT, THE LARGE SCALES (small k values) in the power spectra are not physical
import warnings
warnings.filterwarnings('ignore')

fig, axs = plt.subplots(2, 2, figsize=(18,15))
for ax in axs.flatten():
    ax.tick_params(axis='both',which='both',direction='in')
fig.suptitle('Linear Matter Power Spectra')

cosmosis_output_dir = 'chains/case_B/cosmosis_output/matter_power_lin/'
ks = like.cosmo.cosmo_dic['k_win']
h = values['cosmological_parameters']['h0']

axs[0,0].loglog(ks, like.cosmo.cosmo_dic['Pk_delta'].P(like.cosmo.cosmo_dic['z_win'][0], ks), 'y-', 
           label=r"$z = {:.2f}$ Benchmark".format(like.cosmo.cosmo_dic['z_win'][0]))
axs[0,0].loglog(np.loadtxt(cosmosis_output_dir+'k_h.txt')*h, np.loadtxt(cosmosis_output_dir+'p_k.txt')[0,:]/h**3, 'k:', 
           label='CosmoSIS')
axs[0,0].set_xlabel(k_label)
axs[0,0].set_ylabel(pm_label)
axs[0,0].legend(frameon=False)

axs[0,1].loglog(ks, like.cosmo.cosmo_dic['Pk_delta'].P(like.cosmo.cosmo_dic['z_win'][25], ks), 'y-', 
           label=r"$z = {:.2f}$ Benchmark".format(like.cosmo.cosmo_dic['z_win'][25]))
axs[0,1].loglog(np.loadtxt(cosmosis_output_dir+'k_h.txt')*h, np.loadtxt(cosmosis_output_dir+'p_k.txt')[25,:]/h**3, 'k:', 
           label='CosmoSIS')
axs[0,1].set_xlabel(k_label)
axs[0,1].set_ylabel(pm_label)
axs[0,1].legend(frameon=False)

axs[1,0].loglog(ks, like.cosmo.cosmo_dic['Pk_delta'].P(like.cosmo.cosmo_dic['z_win'][50], ks), 'y-', 
           label=r"$z = {:.2f}$ Benchmark".format(like.cosmo.cosmo_dic['z_win'][50]))
axs[1,0].loglog(np.loadtxt(cosmosis_output_dir+'k_h.txt')*h, np.loadtxt(cosmosis_output_dir+'p_k.txt')[50,:]/h**3, 'k:', 
           label='CosmoSIS')
axs[1,0].set_xlabel(k_label)
axs[1,0].set_ylabel(pm_label)
axs[1,0].legend(frameon=False)

#------
axs[1,1].loglog(ks, like.cosmo.cosmo_dic['Pk_delta'].P(like.cosmo.cosmo_dic['z_win'][-1], ks), 'y-', 
           label=r"$z = {:.2f}$ Benchmark".format(like.cosmo.cosmo_dic['z_win'][-1]))
axs[1,1].loglog(np.loadtxt(cosmosis_output_dir+'k_h.txt')*h, np.loadtxt(cosmosis_output_dir+'p_k.txt')[-1,:]/h**3, 'k:', 
           label='CosmoSIS')
axs[1,1].set_xlabel(k_label)
axs[1,1].set_ylabel(pm_label)
axs[1,1].legend(frameon=False)

In [ ]:
# Let's plot the matter power spectrum 
k_label           = r'$k$ $[\rm{Mpc}^{-1}]$'
pm_label          = r'$P_{\delta\delta} \left[\rm{Mpc}^3\right]$'

# WARNING: AT THE MOMENT, THE LARGE SCALES (small k values) in the power spectra are not physical
import warnings
warnings.filterwarnings('ignore')

fig, axs = plt.subplots(2, 2, figsize=(18,15))
for ax in axs.flatten():
    ax.tick_params(axis='both',which='both',direction='in')
fig.suptitle('Linear Power Spectra (CDM + Baryons)')

cosmosis_output_dir = 'chains/case_B/cosmosis_output/cdm_baryon_power_lin/'
ks = like.cosmo.cosmo_dic['k_win']
h = values['cosmological_parameters']['h0']

axs[0,0].loglog(ks, like.cosmo.cosmo_dic['Pk_cb'].P(like.cosmo.cosmo_dic['z_win'][0], ks), 'y-', 
           label=r"$z = {:.2f}$ Benchmark".format(like.cosmo.cosmo_dic['z_win'][0]))
axs[0,0].loglog(np.loadtxt(cosmosis_output_dir+'k_h.txt')*h, np.loadtxt(cosmosis_output_dir+'p_k.txt')[0,:]/h**3, 'k:', 
           label='CosmoSIS')
axs[0,0].set_xlabel(k_label)
axs[0,0].set_ylabel(pm_label)
axs[0,0].legend(frameon=False)

axs[0,1].loglog(ks, like.cosmo.cosmo_dic['Pk_cb'].P(like.cosmo.cosmo_dic['z_win'][25], ks), 'y-', 
           label=r"$z = {:.2f}$ Benchmark".format(like.cosmo.cosmo_dic['z_win'][25]))
axs[0,1].loglog(np.loadtxt(cosmosis_output_dir+'k_h.txt')*h, np.loadtxt(cosmosis_output_dir+'p_k.txt')[25,:]/h**3, 'k:', 
           label='CosmoSIS')
axs[0,1].set_xlabel(k_label)
axs[0,1].set_ylabel(pm_label)
axs[0,1].legend(frameon=False)

axs[1,0].loglog(ks, like.cosmo.cosmo_dic['Pk_cb'].P(like.cosmo.cosmo_dic['z_win'][50], ks), 'y-', 
           label=r"$z = {:.2f}$ Benchmark".format(like.cosmo.cosmo_dic['z_win'][50]))
axs[1,0].loglog(np.loadtxt(cosmosis_output_dir+'k_h.txt')*h, np.loadtxt(cosmosis_output_dir+'p_k.txt')[50,:]/h**3, 'k:', 
           label='CosmoSIS')
axs[1,0].set_xlabel(k_label)
axs[1,0].set_ylabel(pm_label)
axs[1,0].legend(frameon=False)

#------
axs[1,1].loglog(ks, like.cosmo.cosmo_dic['Pk_cb'].P(like.cosmo.cosmo_dic['z_win'][-1], ks), 'y-', 
           label=r"$z = {:.2f}$ Benchmark".format(like.cosmo.cosmo_dic['z_win'][-1]))
axs[1,1].loglog(np.loadtxt(cosmosis_output_dir+'k_h.txt')*h, np.loadtxt(cosmosis_output_dir+'p_k.txt')[-1,:]/h**3, 'k:', 
           label='CosmoSIS')
axs[1,1].set_xlabel(k_label)
axs[1,1].set_ylabel(pm_label)
axs[1,1].legend(frameon=False)

In [ ]:
# Let's plot the matter power spectrum 
k_label           = r'$k$ $[\rm{Mpc}^{-1}]$'
pm_label          = r'$P_{\delta\delta} \left[\rm{Mpc}^3\right]$'

# WARNING: AT THE MOMENT, THE LARGE SCALES (small k values) in the power spectra are not physical
import warnings
warnings.filterwarnings('ignore')

fig, axs = plt.subplots(2, 2, figsize=(18,15))
for ax in axs.flatten():
    ax.tick_params(axis='both',which='both',direction='in')
fig.suptitle('Linear Weyl Power Spectra')

cosmosis_output_dir = 'chains/case_B/cosmosis_output/weyl_curvature_power_lin/'
ks = like.cosmo.cosmo_dic['k_win']
h = values['cosmological_parameters']['h0']

axs[0,0].loglog(ks, like.cosmo.cosmo_dic['Pk_weyl'].P(like.cosmo.cosmo_dic['z_win'][0], ks), 'y-', 
           label=r"$z = {:.2f}$ Benchmark".format(like.cosmo.cosmo_dic['z_win'][0]))
axs[0,0].loglog(np.loadtxt(cosmosis_output_dir+'k_h.txt')*h, np.loadtxt(cosmosis_output_dir+'p_k.txt')[0,:]/h**3, 'k:', 
           label='CosmoSIS')
axs[0,0].set_xlabel(k_label)
axs[0,0].set_ylabel(pm_label)
axs[0,0].legend(frameon=False)

axs[0,1].loglog(ks, like.cosmo.cosmo_dic['Pk_weyl'].P(like.cosmo.cosmo_dic['z_win'][25], ks), 'y-', 
           label=r"$z = {:.2f}$ Benchmark".format(like.cosmo.cosmo_dic['z_win'][25]))
axs[0,1].loglog(np.loadtxt(cosmosis_output_dir+'k_h.txt')*h, np.loadtxt(cosmosis_output_dir+'p_k.txt')[25,:]/h**3, 'k:', 
           label='CosmoSIS')
axs[0,1].set_xlabel(k_label)
axs[0,1].set_ylabel(pm_label)
axs[0,1].legend(frameon=False)

axs[1,0].loglog(ks, like.cosmo.cosmo_dic['Pk_weyl'].P(like.cosmo.cosmo_dic['z_win'][50], ks), 'y-', 
           label=r"$z = {:.2f}$ Benchmark".format(like.cosmo.cosmo_dic['z_win'][50]))
axs[1,0].loglog(np.loadtxt(cosmosis_output_dir+'k_h.txt')*h, np.loadtxt(cosmosis_output_dir+'p_k.txt')[50,:]/h**3, 'k:', 
           label='CosmoSIS')
axs[1,0].set_xlabel(k_label)
axs[1,0].set_ylabel(pm_label)
axs[1,0].legend(frameon=False)

#------
axs[1,1].loglog(ks, like.cosmo.cosmo_dic['Pk_weyl'].P(like.cosmo.cosmo_dic['z_win'][-1], ks), 'y-', 
           label=r"$z = {:.2f}$ Benchmark".format(like.cosmo.cosmo_dic['z_win'][-1]))
axs[1,1].loglog(np.loadtxt(cosmosis_output_dir+'k_h.txt')*h, np.loadtxt(cosmosis_output_dir+'p_k.txt')[-1,:]/h**3, 'k:', 
           label='CosmoSIS')
axs[1,1].set_xlabel(k_label)
axs[1,1].set_ylabel(pm_label)
axs[1,1].legend(frameon=False)